In [1]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

## Open file
fp = open('doc_list.txt','r')
doc_list = [ ]
line = fp.readline()

## 用 while 逐行讀取檔案內容，直至檔案結尾
while line:
    line = line.replace('\n','')
    doc_list.append(line)
    line = fp.readline()
 
fp.close()
print(len(doc_list))

## Open file
fp = open('query_list.txt','r')
query_list = [ ]
line = fp.readline()

## 用 while 逐行讀取檔案內容，直至檔案結尾
while line:
    line = line.replace('\n','')
    query_list.append(line)
    line = fp.readline()
 
fp.close()
print(len(query_list))

voc = [ ]

2265
800


In [2]:
# Open file
doc_len = 0
doc_temp = [ ]
doc = [ ]

while doc_len < len(doc_list) :
    fp = open('Document/'+doc_list[doc_len],'r')
    i = 0
    while i < 3:
        buffer = fp.readline()
        i += 1
    
    doc_temp = fp.read()
    doc_temp = doc_temp.replace('-1','')
    doc_temp = doc_temp.split()
    doc = doc + doc_temp
    doc = list(set(doc))

    fp.close()
    doc_len += 1
    
print(len(doc))

13290


In [3]:
# Open file
query_len = 0
query_temp = [ ]
query = [ ]

while query_len < len(query_list) :
    fp = open('Query/'+query_list[query_len],'r')
    
    query_temp = fp.read()
    query_temp = query_temp.replace('-1','')
    query_temp = query_temp.split()
    query = query + query_temp
    #word = word + query_temp
    query = list(set(query))
    
    fp.close()
    query_len += 1
    
print(len(query))

9735


In [4]:
voc = doc + query
voc = list(set(voc))
print(len(voc))

15884


In [5]:
tfd = np.zeros( (len(voc), doc_len), dtype=np.int )
#print(tfd)
tfq = np.zeros( (len(voc), query_len), dtype=np.int )
#print(tfq)

In [6]:
x = 0
temp = [ ]

while x < doc_len :
    fp = open('Document/'+doc_list[x],'r')
    
    i = 0
    while i < 3:
        buffer = fp.readline()
        i += 1
    
    temp = fp.read()
    temp = temp.replace('-1','')
    temp = temp.split()

    y = 0
    for w in voc :
        for dc in temp :
            if w == dc :
                tfd[y][x] += 1
        y += 1
        
    fp.close()
    x += 1
    
print(tfd)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:
#print(np.unique(tfd, axis=0))

In [8]:
x = 0
while x < query_len :
    fp = open('Query/'+query_list[x],'r')
    
    i = 0
    while i < 3:
        buffer = fp.readline()
        i += 1
    
    temp = fp.read()
    temp = temp.replace('-1','')
    temp = temp.split()

    y = 0
    for w in voc :
        for q in temp :
            if w == q :
                tfq[y][x] += 1
        y += 1
        
    fp.close()
    x += 1
    
print(tfq)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
#print(np.unique(tfq, axis=0))
for i in range(len(voc)) :
    for j in range(doc_len) :
        if tfd[i][j] > 0 :
            tfd[i][j] = math.log(tfd[i][j],2) + 1
            
for i in range(len(voc)) :
    for j in range(query_len) :
        if tfq[i][j] > 0 :
            tfq[i][j] = math.log(tfq[i][j],2) + 1

In [10]:
idf = np.zeros( (len(voc), 1 ), dtype=np.float )
#print(idf)

for i in range(len(voc)) :
    for j in range(doc_len) :
        if tfd[i][j] > 0 :
            idf[i][0] += 1

print(idf.shape)
#print(np.unique(idf))

for i in range(len(voc)) :
    if idf[i][0] != 0 :
        idf[i][0] = doc_len / idf[i][0]
        idf[i][0] = math.log(idf[i][0],10) 
        
print(idf)

(15884, 1)
[[2.87794695]
 [0.        ]
 [2.87794695]
 ...
 [3.35506821]
 [3.35506821]
 [2.75300822]]


In [11]:
tfidfd = [ ]
tfidfd = np.zeros( (len(voc), doc_len), dtype=np.float )
print(tfidfd.shape)

tfidfq = [ ]
tfidfq = np.zeros( (len(voc), query_len), dtype=np.float )
print(tfidfq.shape)


(15884, 2265)
(15884, 800)


In [12]:
for i in range(len(voc)) :
    for j in range(doc_len) :
        tfidfd[i][j] = tfd[i][j] * idf[i][0]
print(np.unique(tfidfd))        

for i in range(len(voc)) :
    for j in range(query_len) :
        tfidfq[i][j] = tfq[i][j] * idf[i][0]
print(np.unique(tfidfq))

[0.00000000e+00 9.59767201e-04 1.91953440e-03 ... 1.52701911e+01
 1.67753410e+01 1.72676817e+01]
[0.00000000e+00 9.59767201e-04 1.91953440e-03 ... 1.52701911e+01
 1.67753410e+01 1.83242293e+01]


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

vsm_1 = [ ]
vsm_1 = np.zeros( (query_len, doc_len), dtype=np.float )
print(vsm_1.shape)

(800, 2265)


In [14]:
#vsm_1 = cosine_similarity(tfidfd.T, tfidfq.T)

for i in range(query_len) :
    for j in range(doc_len) :
        vsm_1[i][j] = cosine_similarity([tfidfd[:,j]], [tfidfq[:,i]])

#vsm_1 = vsm_1.T
print(vsm_1.shape)

(800, 2265)


In [15]:
ranking = pd.DataFrame(vsm_1, index = query_list, columns = doc_list)
#print(ranking)


In [16]:
ranking = ranking.sort_values(by = query_list[0],ascending= False, axis=1)
ranking

,VOM19980630.0730.0191,VOM19980630.0700.0238,VOM19980630.0900.0105,VOM19980604.0700.0157,VOM19980604.0730.0077,VOM19980221.0700.2442,VOM19980302.0700.1233,VOM19980319.0900.2928,VOM19980228.0700.2757,VOM19980524.0730.0031,...,VOM19980612.0730.0286,VOM19980526.0700.0538,VOM19980521.0700.0559,VOM19980610.0700.0539,VOM19980424.0900.0217,VOM19980529.0700.0562,VOM19980529.0900.0276,VOM19980421.0900.0213,VOM19980327.0900.3495,VOM19980626.0700.0539
40001.query,0.210643,0.210643,0.210143,0.167003,0.158475,0.158465,0.154157,0.144558,0.140445,0.131900,...,0.000098,0.000075,2.808290e-05,2.276685e-05,0.000022,2.116511e-05,2.116511e-05,2.627716e-08,2.470519e-08,2.064391e-08
40002.query,0.006827,0.006827,0.006811,0.028443,0.025549,0.036364,0.054439,0.057501,0.046870,0.038254,...,0.007622,0.002024,4.945394e-03,7.280493e-03,0.014860,2.332932e-03,2.332932e-03,1.143362e-02,6.553201e-03,2.462062e-02
40003.query,0.021220,0.021220,0.021170,0.039169,0.040863,0.037467,0.034225,0.043582,0.026557,0.006264,...,0.014729,0.009445,1.937515e-02,6.047919e-03,0.018912,1.383567e-02,1.383567e-02,4.499932e-03,3.276843e-02,2.282947e-02
40004.query,0.036927,0.036927,0.036840,0.050131,0.037121,0.067717,0.062145,0.093812,0.046678,0.015039,...,0.012951,0.010849,1.429538e-02,5.722167e-03,0.009589,3.017142e-02,3.017142e-02,1.833328e-02,1.949135e-08,1.597064e-02
40005.query,0.001957,0.001957,0.001952,0.018757,0.011809,0.008841,0.022558,0.044683,0.011812,0.000859,...,0.004975,0.005595,4.214756e-02,2.581321e-05,0.008657,2.566579e-02,2.566579e-02,9.022222e-04,2.801092e-08,2.340622e-08
40006.query,0.182941,0.182941,0.182506,0.087341,0.077822,0.179826,0.068416,0.091507,0.159191,0.173268,...,0.004930,0.002088,3.631643e-05,1.773619e-03,0.009625,2.736609e-05,2.736609e-05,1.306150e-02,1.078779e-02,6.950704e-03
40007.query,0.154912,0.154912,0.154544,0.120450,0.133508,0.138028,0.114782,0.112963,0.137703,0.151060,...,0.004431,0.021582,2.717273e-03,8.947994e-03,0.008761,3.227430e-02,3.227430e-02,1.755544e-02,8.795362e-03,1.290885e-02
40008.query,0.020147,0.020147,0.020099,0.040069,0.043867,0.023001,0.038410,0.034465,0.032152,0.029265,...,0.009651,0.009890,4.172758e-02,4.959940e-03,0.014351,1.123828e-02,1.123828e-02,1.706836e-02,1.473532e-02,1.892939e-02
40009.query,0.021911,0.021911,0.021859,0.031649,0.031783,0.019133,0.047489,0.043420,0.030827,0.018473,...,0.004869,0.012467,2.346396e-03,6.156050e-03,0.004354,2.376788e-02,2.376788e-02,3.810317e-02,1.416658e-03,8.655812e-04
40010.query,0.019696,0.019696,0.019649,0.035948,0.031327,0.025048,0.030310,0.032388,0.020866,0.008822,...,0.013364,0.013213,5.419074e-05,3.656157e-04,0.004391,4.297560e-02,4.297560e-02,4.706689e-03,3.057770e-02,1.500169e-02


In [25]:
Rq = 5

alpha = 0.3
alpha_idf = np.zeros( (len(voc), 1 ), dtype=np.float )

beta = 1.1
beta_idf = np.zeros( (len(voc), 1 ), dtype=np.float )

new_idf = np.zeros( (len(voc), 1 ), dtype=np.float )

tfidfq_2 = np.zeros( (len(voc), query_len), dtype=np.float )
print(tfidfq_2.shape)

for n in range(query_len) :
    ranking = ranking.sort_values(by = query_list[n],ascending= False, axis=1)
    name = ranking.columns
    name = np.asarray(name)

    #抓出原本的tfidf_doc

    for i in range(Rq) :
        for j in range(doc_len) :
            if name[i] == doc_list[j]:
                #print(name[i], j)
                beta_idf = beta_idf[:,[0]] + tfidfd[:,[j]]
            
    alpha_idf = alpha_idf[:,[0]] + tfidfq[:,[n]]
    alpha_idf = alpha_idf * alpha
    
    beta_idf = beta_idf * beta * (1/Rq)

    new_idf = alpha_idf + beta_idf
    #print(alpha_idf, beta_idf, new_idf)

    tfidfq_2[:,[n]] = new_idf
    #print(tfidfq_2[:,[n]])

(15884, 800)


In [26]:
print(tfidfq_2)

[[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 2.36665238e-042
  5.20663524e-043 1.14545975e-043]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 ...
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 4.87803126e-110
  1.46340938e-110 4.39022813e-111]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]]


In [27]:
print(tfidfd.shape)
print(tfidfq_2.shape)
vsm_2 = [ ]
vsm_2 = np.zeros( (query_len, doc_len), dtype=np.float )
print(vsm_2.shape)

(15884, 2265)
(15884, 800)
(800, 2265)


In [28]:
#vsm_2 = cosine_similarity(tfidfd.T, tfidfq_2.T)
#print(vsm_2[0][0])
for i in range(query_len) :
    for j in range(doc_len) :
        vsm_2[i][j] = cosine_similarity([tfidfd[:,j]], [tfidfq_2[:,i]])
#         print(vsm_2[i][j])
#         input()

#vsm_2 = vsm_2.T
print(vsm_2)
print(vsm_2.shape)

[[0.01605429 0.01632827 0.01816669 ... 0.01289178 0.01339649 0.00661442]
 [0.09834463 0.05363987 0.0516624  ... 0.050806   0.03905569 0.03431305]
 [0.06635962 0.03883987 0.04346299 ... 0.04595465 0.03957829 0.03209139]
 ...
 [0.02923153 0.04555753 0.04242503 ... 0.02615071 0.02140626 0.0293437 ]
 [0.06107942 0.06110583 0.04535251 ... 0.04822187 0.04778577 0.04658813]
 [0.04996628 0.04834675 0.04959038 ... 0.05785977 0.05114251 0.06200297]]
(800, 2265)


In [29]:
ranking = pd.DataFrame(vsm_2, index = query_list, columns = doc_list)
#print(ranking)


In [30]:
f = open('M10715086.txt','w') 
f.write("Query,RetrievedDocuments\n")
f.close()

f = open('M10715086.txt','a') 

for i in range(query_len) :
    f.write(query_list[i] + ',')
    ranking = ranking.sort_values(by = query_list[i],ascending= False, axis=1)
    name = ranking.columns
    name = np.asarray(name)
    for j in range(doc_len):
        f.write(name[j] + ' ')
    f.write('\n')

f.close()